In [13]:
#%matplotlib inline 
import torch 
#from IPython import display 
#from matplotlib import pyplot as plt
import numpy as np 
import random
import math
from time import time
#from torch import optim

In [2]:
def generatedata(noise,showpermutation=False,showtrue_w=False):
    true_w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if showtrue_w:
        print('true_w2:',true_w2)
    X2_before_ =torch.from_numpy(np.random.normal(0, 1, (num_example, num_X2feature)))
    y_ = torch.mm(X2_before_,true_w2)
    y_ += torch.from_numpy(np.random.normal(0, noise ,size=y_.size()))
    P_array=np.random.permutation(num_example)
    P=torch.zeros(num_example,num_example,dtype=torch.float64)
    for i in range(num_example):
        P[i][P_array[i]]=1
    if showpermutation:
        print('打乱X2的置换矩阵为',P)
    X2_=torch.mm(P,X2_before_)
    #X2_=X2_before_
    error_reg=(torch.norm(y_-torch.mm(X2_before_,true_w2))/torch.norm(y_))
    return y_,X2_,true_w2,P,error_reg

In [3]:
def generateinitialw(method,showinitialw=False):
    if method=='normal':
        w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if method=='zeros':
        w2=torch.zeros(num_X2feature,1,dtype=torch.float64)
    if showinitialw:
        print('initial w2:',w2)
    return w2

In [4]:
def sinkhorn_stabilized(a, b, M, reg, numItermax=1000, tau=1e3, stopThr=1e-9,
                        warmstart=None, verbose=False, print_period=20,
                        log=False, **kwargs):

#     a = np.asarray(a, dtype=np.float64)
#     b = np.asarray(b, dtype=np.float64)
#     M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M

#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # test if multiple target
#     if len(b.shape) > 1:
#         n_hists = b.shape[1]
#         a = a[:, np.newaxis]
#     else:
#         n_hists = 0
    n_hists = 0
    # init data
    dim_a = len(a)
    dim_b = len(b)

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    if n_hists:
        u = torch.ones((dim_a, n_hists)) / dim_a
        v = torch.ones((dim_b, n_hists)) / dim_b
    else:
        u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))- beta.reshape((1, dim_b))) / reg)

    def get_Gamma(alpha, beta, u, v):
        """log space gamma computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1)) - beta.reshape((1, dim_b)))
                      / reg + torch.log(u.reshape((dim_a, 1))) + torch.log(v.reshape((1, dim_b))))

    # print(np.min(K))

    K = get_K(alpha, beta)
    transp = K
    loop = 1
    cpt = 0
    err = 1
    while loop:

        uprev = u
        vprev = v
        # sinkhornrn update
        v = b / (torch.mm(K.transpose(1,0), u) + 1e-16)
        u = a / (torch.mm(K, v) + 1e-16)
        # remove numerical problems and store them in K
        if torch.abs(u).max() > tau or torch.abs(v).max() > tau:
            if n_hists:
                alpha, beta = alpha + reg * \
                    torch.max(torch.log(u), 1), beta + reg * torch.max(np.log(v))
            else:
                alpha, beta = alpha + reg * torch.log(u), beta + reg * torch.log(v)
                if n_hists:
                    u, v = torch.ones((dim_a, n_hists)) / dim_a, torch.ones((dim_b, n_hists)) / dim_b
                else:
                    u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b
            K = get_K(alpha, beta)
            

        if cpt % print_period == 0:
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            if n_hists:
                err_u = abs(u - uprev).max()
                err_u /= max(abs(u).max(), abs(uprev).max(), 1.)
                err_v = abs(v - vprev).max()
                err_v /= max(abs(v).max(), abs(vprev).max(), 1.)
                err = 0.5 * (err_u + err_v)
            else:
                transp = get_Gamma(alpha, beta, u, v)
                err = torch.norm((torch.sum(transp, axis=0) - b))
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 20) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr:
            loop = False

        if cpt >= numItermax:
            loop = False

        if np.any(np.isnan(u.detach().numpy())) or np.any(np.isnan(v.detach().numpy())):
            # we have reached the machine precision
            # come back to previous solution and quit loop
            print('Warning: numerical errors at iteration', cpt)
            u = uprev
            v = vprev
            break

        cpt = cpt + 1
    #print(cpt)
    if log:
        if n_hists:
            alpha = alpha[:, None]
            beta = beta[:, None]
        logu = alpha / reg + torch.log(u)
        logv = beta / reg + torch.log(v)
        log['logu'] = logu
        log['logv'] = logv
        log['alpha'] = alpha + reg * torch.log(u)
        log['beta'] = beta + reg * torch.log(v)
        log['warmstart'] = (log['alpha'], log['beta'])
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res, log

        else:
            return get_Gamma(alpha, beta, u, v), log
    else:
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res
        else:
            return get_Gamma(alpha, beta, u, v)

In [5]:
def sinkhorn_epsilon_scaling(a, b, M, reg, numItermax=100, epsilon0=1e4,
                             numInnerItermax=100, tau=1e3, stopThr=1e-9,
                             warmstart=None, verbose=False, print_period=10,
                             log=False, **kwargs):
    #a = np.asarray(a, dtype=np.float64)
    #b = np.asarray(b, dtype=np.float64)
    #M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M
#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # init data
    dim_a = len(a)
    #dim_a=num_example
    dim_b = len(b)
    #dim_b=num_example
    # nrelative umerical precision with 64 bits
    numItermin = 35
    numItermax = max(numItermin, numItermax)  # ensure that last velue is exact

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))
                        - beta.reshape((1, dim_b))) / reg)

    # print(np.min(K))
    def get_reg(n):  # exponential decreasing
        return (epsilon0 - reg) * np.exp(-n) + reg

    loop = 1
    cpt = 0
    err = 1
    while loop:

        regi = get_reg(cpt)

        G, logi = sinkhorn_stabilized(a, b, M, regi,
                                      numItermax=numInnerItermax, stopThr=1e-9,
                                      warmstart=(alpha, beta), verbose=False,
                                      print_period=20, tau=tau, log=True)

        alpha = logi['alpha']
        beta = logi['beta']

        if cpt >= numItermax:
            loop = False

        if cpt % (print_period) == 0:  # spsion nearly converged
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            transp = G
            err = torch.norm(
                (torch.sum(transp, axis=0) - b))**2 + torch.norm((torch.sum(transp, axis=1) - a))**2
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 10) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr and cpt > numItermin:
            loop = False

        cpt = cpt + 1
    # print('err=',err,' cpt=',cpt)
    if log:
        log['alpha'] = alpha
        log['beta'] = beta
        log['warmstart'] = (log['alpha'], log['beta'])
        return G, log
    else:
        return G
    

In [40]:
gama_=1
eta=0
starts=1
for i_____ in range(5):
    for num_example in range(300,301,100): 
        for num_X2feature in [30]:
            for i____ in range(10):
                (y_,X2_,true_w2,true_P,error_reg1)=generatedata(noise=0.3)
                y=y_
                X2=X2_
                results_Loss = []
                results_w2=[]
                results_error=[]
                for i__ in range(starts):
    #                     P_array=np.random.permutation(num_example)
    #                     P=torch.zeros(num_example,num_example,dtype=torch.float64)
    #                     for i in range(num_example):
    #                         P[i][P_array[i]]=1
    #                     X_=torch.cat([X1,X2],1)
    #                     X=torch.mm(P,X_)
    #                     w=torch.mm(torch.mm(torch.tensor(np.linalg.inv(torch.mm(X.transpose(1,0),X))),X.transpose(1,0)),y)
    #                     w1,w2=w.split([num_X1feature,num_X2feature],dim=0)
    #                     w1=torch.from_numpy(np.random.normal(0, 0,(num_X1feature,1)))
    #                     w2=torch.from_numpy(np.random.normal(0, 0,(num_X2feature,1)))
                    w2=generateinitialw(method='zeros')
                    #w2=true_w2
                    w2.requires_grad_(requires_grad=True)
    #                 results_Loss = []
                    lr=0.0016
                    results_S=[]
                    t=0
                    before1=0
                    while True:
                        #start=time()
                        Y1=y
                        Y2=torch.mm(X2,w2)
                        C=torch.zeros(num_example,num_example,dtype=torch.float64)
                        for i in range(num_example):
                            for j in range(num_example):
                                C[i][j]=(Y1[i]-Y2[j])**2            

                        #S=SinkhornIPOT(C)
                        a=torch.ones(num_example,1,dtype=torch.float64)
                        b=torch.ones(num_example,1,dtype=torch.float64)
                        S=sinkhorn_epsilon_scaling(a, b, C, 0.00000001)
                        #print(S.transpose(1,0).half())
                        #results_S.append(S)
                        #if t>0:
                            #print('        S变化',(torch.norm(results_S[t]-results_S[t-1]))/(torch.norm(results_S[t-1])))
                        #Loss=torch.sum(S*C)
                        Loss=torch.norm(Y1-torch.mm(S,Y2))**2
                        if Loss<1e-2:
                            break
                        Loss.backward()
    #                         results_Loss.append(Loss)
    #                         for i_ in range(num_X1features):
    #                             results_w1[t][i_]=(w1[i_].data)
    #                         for i_ in range(num_X2features):
    #                             results_w2[t][i_]=(w2[i_].data)
                        w2.data-=lr*(w2.grad+np.random.normal(0,np.sqrt(eta/(1+t)**gama_)))
                        #print(w2.grad)
    #                     if t==num_epochs-1:
    #                         print('最终w1梯度：',w1.grad)
    #                         print('最终w2梯度：',w2.grad)
                        w2.grad.data.zero_() 
                        print('Loss',t,'=',Loss.data)
    #                     if t%6==0:
    #                         if torch.norm(Loss-before1)<1e-4:
    #                             break
    #                         before1=Loss
                        if torch.norm(Loss-before1)/before1<1e-7:
                            break
                        before1=Loss
                        if t>=200:
                            print('超过迭代上限')
                            break
                        if math.isnan(Loss):
                            break
                        t+=1
                        print('#',end='')
                        #print(time()-start)



                    print(' ',end='')
                    error_each=(torch.norm(w2-true_w2))
                    results_error.append(error_each)
                    #results_Loss.append(Loss)
                    #results_w1.append(w1.data)
                    #results_w2.append(w2.data)


                #w1=results_w1[results_Loss.index(min(results_Loss))]
                #w2=results_w2[results_Loss.index(min(results_Loss))]

    #                     for i_ in range(starts):
    #                         results_w1[i_]=(w1[i_].data)
    #                     for i_ in range(starts):
    #                         results_w2[i_]=(w2[i_].data)


                #error_w=((torch.norm(w1-true_w1))/(torch.norm(true_w1))+(torch.norm(w2-true_w2))/(torch.norm(true_w2)))/2
                #print(num_X1feature,num_X2feature,num_example,'平均相对误差1：',error_w)
                print(num_X2feature,num_example,'平均相对误差2：',np.min(results_error),end='   ')

                #print('真实置换矩阵为：',true_P)
                error_P=(torch.norm(S.transpose(1,0)-true_P))/(torch.norm(true_P))
                print(num_X2feature,num_example,'置换矩阵误差：',error_P,end='   ')
                error_reg2=(torch.norm(y_-torch.mm(torch.mm(S,X2_),w2))/torch.norm(y_))
                print('真实回归误差',error_reg1,end='   ')
                print('实验回归误差',error_reg2)
                #print('双随机矩阵S为：',S.transpose(1,0).half())
                #print(results)
    #                 plt.figure(figsize=(6,6))
    #                 plt.plot(results_w1_0,results_Loss, '-o',label='$w1[0]$')
    #                 plt.plot(results_w1_1,results_Loss, '-o',label='$w1[1]$')
    #                 plt.plot(results_w1_2,results_Loss, '-o',label='$w1[2]$')
    #                 plt.plot(results_w2_0,results_Loss, '-o',label='$w2[0]$')
    #                 plt.plot(results_w2_1,results_Loss, '-o',label='$w2[1]$')
    #                 plt.plot(results_w2_2,results_Loss, '-o',label='$w2[2]$')
    #                 plt.legend()
    #                 plt.show()

Loss 0 = tensor(6436.7081, dtype=torch.float64)
#Loss 1 = tensor(6395.9361, dtype=torch.float64)
#Loss 2 = tensor(598.4297, dtype=torch.float64)
#Loss 3 = tensor(523.8526, dtype=torch.float64)
#Loss 4 = tensor(482.5247, dtype=torch.float64)
#Loss 5 = tensor(443.3002, dtype=torch.float64)
#Loss 6 = tensor(407.7794, dtype=torch.float64)
#Loss 7 = tensor(376.8144, dtype=torch.float64)
#Loss 8 = tensor(347.6016, dtype=torch.float64)
#Loss 9 = tensor(320.5688, dtype=torch.float64)
#Loss 10 = tensor(294.2892, dtype=torch.float64)
#Loss 11 = tensor(272.2351, dtype=torch.float64)
#Loss 12 = tensor(250.7513, dtype=torch.float64)
#Loss 13 = tensor(231.5020, dtype=torch.float64)
#Loss 14 = tensor(213.5727, dtype=torch.float64)
#Loss 15 = tensor(195.8759, dtype=torch.float64)
#Loss 16 = tensor(178.3521, dtype=torch.float64)
#Loss 17 = tensor(163.5837, dtype=torch.float64)
#Loss 18 = tensor(149.6106, dtype=torch.float64)
#Loss 19 = tensor(137.5745, dtype=torch.float64)
#Loss 20 = tensor(128.4794, d

#Loss 24 = tensor(70.8115, dtype=torch.float64)
#Loss 25 = tensor(65.5792, dtype=torch.float64)
#Loss 26 = tensor(61.7660, dtype=torch.float64)
#Loss 27 = tensor(58.2490, dtype=torch.float64)
#Loss 28 = tensor(52.9717, dtype=torch.float64)
#Loss 29 = tensor(49.6391, dtype=torch.float64)
#Loss 30 = tensor(47.3152, dtype=torch.float64)
#Loss 31 = tensor(45.2921, dtype=torch.float64)
#Loss 32 = tensor(43.2074, dtype=torch.float64)
#Loss 33 = tensor(40.4937, dtype=torch.float64)
#Loss 34 = tensor(37.8427, dtype=torch.float64)
#Loss 35 = tensor(35.7775, dtype=torch.float64)
#Loss 36 = tensor(34.2677, dtype=torch.float64)
#Loss 37 = tensor(32.7305, dtype=torch.float64)
#Loss 38 = tensor(31.0625, dtype=torch.float64)
#Loss 39 = tensor(29.2167, dtype=torch.float64)
#Loss 40 = tensor(28.0311, dtype=torch.float64)
#Loss 41 = tensor(26.9382, dtype=torch.float64)
#Loss 42 = tensor(25.6841, dtype=torch.float64)
#Loss 43 = tensor(24.6052, dtype=torch.float64)
#Loss 44 = tensor(23.1673, dtype=torch.f

#Loss 96 = tensor(29.4741, dtype=torch.float64)
#Loss 97 = tensor(29.4416, dtype=torch.float64)
#Loss 98 = tensor(29.4029, dtype=torch.float64)
#Loss 99 = tensor(29.3466, dtype=torch.float64)
#Loss 100 = tensor(29.2808, dtype=torch.float64)
#Loss 101 = tensor(29.2014, dtype=torch.float64)
#Loss 102 = tensor(29.1190, dtype=torch.float64)
#Loss 103 = tensor(29.0612, dtype=torch.float64)
#Loss 104 = tensor(28.9545, dtype=torch.float64)
#Loss 105 = tensor(28.8575, dtype=torch.float64)
#Loss 106 = tensor(28.7369, dtype=torch.float64)
#Loss 107 = tensor(28.5886, dtype=torch.float64)
#Loss 108 = tensor(28.4402, dtype=torch.float64)
#Loss 109 = tensor(28.2903, dtype=torch.float64)
#Loss 110 = tensor(28.1778, dtype=torch.float64)
#Loss 111 = tensor(28.0694, dtype=torch.float64)
#Loss 112 = tensor(27.9363, dtype=torch.float64)
#Loss 113 = tensor(27.7942, dtype=torch.float64)
#Loss 114 = tensor(27.6883, dtype=torch.float64)
#Loss 115 = tensor(27.5600, dtype=torch.float64)
#Loss 116 = tensor(27.47

#Loss 95 = tensor(7.9916, dtype=torch.float64)
#Loss 96 = tensor(7.9885, dtype=torch.float64)
#Loss 97 = tensor(7.9877, dtype=torch.float64)
#Loss 98 = tensor(7.9875, dtype=torch.float64)
#Loss 99 = tensor(7.9874, dtype=torch.float64)
#Loss 100 = tensor(7.9874, dtype=torch.float64)
#Loss 101 = tensor(7.9874, dtype=torch.float64)
 30 300 平均相对误差2： tensor(8.3027, dtype=torch.float64, grad_fn=<NormBackward0>)   30 300 置换矩阵误差： tensor(1.4119, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0492, dtype=torch.float64)   实验回归误差 tensor(0.0270, dtype=torch.float64, grad_fn=<DivBackward0>)
Loss 0 = tensor(6022.3120, dtype=torch.float64)
#Loss 1 = tensor(5866.2707, dtype=torch.float64)
#Loss 2 = tensor(308.9416, dtype=torch.float64)
#Loss 3 = tensor(253.4899, dtype=torch.float64)
#Loss 4 = tensor(236.1863, dtype=torch.float64)
#Loss 5 = tensor(221.8701, dtype=torch.float64)
#Loss 6 = tensor(209.1237, dtype=torch.float64)
#Loss 7 = tensor(198.1352, dtype=torch.float64)
#Loss 8 = tenso

#Loss 59 = tensor(49.6676, dtype=torch.float64)
#Loss 60 = tensor(49.3491, dtype=torch.float64)
#Loss 61 = tensor(49.0783, dtype=torch.float64)
#Loss 62 = tensor(48.8889, dtype=torch.float64)
#Loss 63 = tensor(48.7710, dtype=torch.float64)
#Loss 64 = tensor(48.6093, dtype=torch.float64)
#Loss 65 = tensor(48.2533, dtype=torch.float64)
#Loss 66 = tensor(48.0805, dtype=torch.float64)
#Loss 67 = tensor(47.8893, dtype=torch.float64)
#Loss 68 = tensor(47.3811, dtype=torch.float64)
#Loss 69 = tensor(47.1046, dtype=torch.float64)
#Loss 70 = tensor(46.8050, dtype=torch.float64)
#Loss 71 = tensor(46.6171, dtype=torch.float64)
#Loss 72 = tensor(46.2457, dtype=torch.float64)
#Loss 73 = tensor(45.8073, dtype=torch.float64)
#Loss 74 = tensor(45.2731, dtype=torch.float64)
#Loss 75 = tensor(44.8335, dtype=torch.float64)
#Loss 76 = tensor(44.4593, dtype=torch.float64)
#Loss 77 = tensor(44.0713, dtype=torch.float64)
#Loss 78 = tensor(43.8288, dtype=torch.float64)
#Loss 79 = tensor(43.5725, dtype=torch.f

...

#Loss 87 = tensor(5.1274, dtype=torch.float64)
#Loss 88 = tensor(5.1243, dtype=torch.float64)
#Loss 89 = tensor(5.1224, dtype=torch.float64)
#Loss 90 = tensor(5.1222, dtype=torch.float64)
#Loss 91 = tensor(5.1215, dtype=torch.float64)
#Loss 92 = tensor(5.1213, dtype=torch.float64)
#Loss 93 = tensor(5.1213, dtype=torch.float64)
#Loss 4 = tensor(723.2613, dtype=torch.float64)
#Loss 5 = tensor(681.9732, dtype=torch.float64)
#Loss 6 = tensor(641.5343, dtype=torch.float64)
#Loss 7 = tensor(604.1947, dtype=torch.float64)
#Loss 8 = tensor(567.9806, dtype=torch.float64)
#Loss 9 = tensor(533.5700, dtype=torch.float64)
#Loss 10 = tensor(499.2694, dtype=torch.float64)
#Loss 11 = tensor(463.6460, dtype=torch.float64)
#Loss 12 = tensor(427.9526, dtype=torch.float64)
#Loss 13 = tensor(392.3084, dtype=torch.float64)
#Loss 14 = tensor(359.0053, dtype=torch.float64)
#Loss 15 = tensor(326.8051, dtype=torch.float64)
#Loss 16 = tensor(296.6282, dtype=torch.float64)
#Loss 17 = tensor(267.7028, dtype=torch.

#Loss 83 = tensor(11.3353, dtype=torch.float64)
#Loss 84 = tensor(11.1643, dtype=torch.float64)
#Loss 85 = tensor(11.0188, dtype=torch.float64)
#Loss 86 = tensor(10.8787, dtype=torch.float64)
#Loss 87 = tensor(10.5923, dtype=torch.float64)
#Loss 88 = tensor(10.3096, dtype=torch.float64)
#Loss 89 = tensor(10.1370, dtype=torch.float64)
#Loss 90 = tensor(10.0096, dtype=torch.float64)
#Loss 91 = tensor(9.9040, dtype=torch.float64)
#Loss 92 = tensor(9.8223, dtype=torch.float64)
#Loss 93 = tensor(9.7634, dtype=torch.float64)
#Loss 94 = tensor(9.7369, dtype=torch.float64)
#Loss 95 = tensor(9.6979, dtype=torch.float64)
#Loss 96 = tensor(9.6478, dtype=torch.float64)
#Loss 97 = tensor(9.6264, dtype=torch.float64)
#Loss 98 = tensor(9.6061, dtype=torch.float64)
#Loss 99 = tensor(9.5413, dtype=torch.float64)
#Loss 100 = tensor(9.4725, dtype=torch.float64)
#Loss 101 = tensor(9.3622, dtype=torch.float64)
#Loss 102 = tensor(9.1872, dtype=torch.float64)
#Loss 103 = tensor(9.0400, dtype=torch.float64)
#

#Loss 113 = tensor(19.2930, dtype=torch.float64)
#Loss 114 = tensor(19.2825, dtype=torch.float64)
#Loss 115 = tensor(19.2769, dtype=torch.float64)
#Loss 116 = tensor(19.2681, dtype=torch.float64)
#Loss 117 = tensor(19.2609, dtype=torch.float64)
#Loss 118 = tensor(19.2577, dtype=torch.float64)
#Loss 119 = tensor(19.2563, dtype=torch.float64)
#Loss 120 = tensor(19.2551, dtype=torch.float64)
#Loss 121 = tensor(19.2550, dtype=torch.float64)
#Loss 122 = tensor(19.2550, dtype=torch.float64)
#Loss 123 = tensor(19.2550, dtype=torch.float64)
#Loss 124 = tensor(19.2550, dtype=torch.float64)
 30 300 平均相对误差2： tensor(6.2552, dtype=torch.float64, grad_fn=<NormBackward0>)   30 300 置换矩阵误差： tensor(1.4119, dtype=torch.float64, grad_fn=<DivBackward0>)   真实回归误差 tensor(0.0641, dtype=torch.float64)   实验回归误差 tensor(0.0573, dtype=torch.float64, grad_fn=<DivBackward0>)
Loss 0 = tensor(8746.9308, dtype=torch.float64)
#Loss 1 = tensor(8328.6036, dtype=torch.float64)
#Loss 2 = tensor(579.6663, dtype=torch.float64

#Loss 24 = tensor(176.8347, dtype=torch.float64)
#Loss 25 = tensor(168.0725, dtype=torch.float64)
#Loss 26 = tensor(160.5893, dtype=torch.float64)
#Loss 27 = tensor(154.8102, dtype=torch.float64)
#Loss 28 = tensor(150.0830, dtype=torch.float64)
#Loss 29 = tensor(144.5480, dtype=torch.float64)
#Loss 30 = tensor(137.6120, dtype=torch.float64)
#Loss 31 = tensor(131.0662, dtype=torch.float64)
#Loss 32 = tensor(124.7288, dtype=torch.float64)
#Loss 33 = tensor(118.4424, dtype=torch.float64)
#Loss 34 = tensor(113.2261, dtype=torch.float64)
#Loss 35 = tensor(108.7405, dtype=torch.float64)
#Loss 36 = tensor(105.4770, dtype=torch.float64)
#Loss 37 = tensor(101.6928, dtype=torch.float64)
#Loss 38 = tensor(97.1948, dtype=torch.float64)
#Loss 39 = tensor(92.4838, dtype=torch.float64)
#Loss 40 = tensor(87.8815, dtype=torch.float64)
#Loss 41 = tensor(84.1832, dtype=torch.float64)
#Loss 42 = tensor(80.5854, dtype=torch.float64)
#Loss 43 = tensor(77.4445, dtype=torch.float64)
#Loss 44 = tensor(75.0243,

KeyboardInterrupt: 